In [1]:
import torch
from transformers import GPTQConfig, AutoTokenizer, AutoModelForSequenceClassification,AutoModelForCausalLM,Gemma2ForCausalLM, BitsAndBytesConfig
import logging
import pandas as pd
import pickle
from datasets import load_from_disk
import os
os.chdir("/group-volume/binfeng/wsdm/stage_qft")
MODEL_PATH = "/group-volume/binfeng/wsdm/ckpt/q8/qwen14b_dare_dslerp_bnb_q8"
MAX_LEN = 3000
MAX_PROMPT_LENGTH = 400

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

/home/user/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# data = pd.read_parquet("/group-volume/binfeng/wsdm/stage_merge/qwen_ft_text.parquet").sample(n=32, random_state=2023)
# data = data["text"].to_list()
# gptq_config = GPTQConfig(bits=8, dataset = data, model_seqlen=3000, group_size = 128,
#                          tokenizer=tokenizer)

# bnb_config =  BitsAndBytesConfig(
#         load_in_4bit = True,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_compute_dtype=torch.bfloat16,
#         bnb_4bit_use_double_quant=True,
#     )

# bnb_config =  BitsAndBytesConfig(
#         load_in_8bit = True
#     )

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    # torch_dtype=torch.bfloat16,
    # num_labels=2,
    device_map='auto',
    # quantization_config=gptq_config,
    )

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Loading checkpoint shards: 100%|██████████| 3/3 [00:25<00:00,  8.57s/it]


In [4]:
for param_name, param in model.named_parameters():
    print(f"{param_name}: {type(param)}")

model.embed_tokens.weight: <class 'torch.nn.parameter.Parameter'>
model.layers.0.self_attn.q_proj.weight: <class 'bitsandbytes.nn.modules.Int8Params'>
model.layers.0.self_attn.q_proj.bias: <class 'torch.nn.parameter.Parameter'>
model.layers.0.self_attn.k_proj.weight: <class 'bitsandbytes.nn.modules.Int8Params'>
model.layers.0.self_attn.k_proj.bias: <class 'torch.nn.parameter.Parameter'>
model.layers.0.self_attn.v_proj.weight: <class 'bitsandbytes.nn.modules.Int8Params'>
model.layers.0.self_attn.v_proj.bias: <class 'torch.nn.parameter.Parameter'>
model.layers.0.self_attn.o_proj.weight: <class 'bitsandbytes.nn.modules.Int8Params'>
model.layers.0.mlp.gate_proj.weight: <class 'bitsandbytes.nn.modules.Int8Params'>
model.layers.0.mlp.up_proj.weight: <class 'bitsandbytes.nn.modules.Int8Params'>
model.layers.0.mlp.down_proj.weight: <class 'bitsandbytes.nn.modules.Int8Params'>
model.layers.0.input_layernorm.weight: <class 'torch.nn.parameter.Parameter'>
model.layers.0.post_attention_layernorm.w

In [ ]:
merged_quantized_model.save_pretrained("/group-volume/binfeng/wsdm/ckpt/q4/qwen14b_dare_dslerp_gptq_q4")
tokenizer.save_pretrained("/group-volume/binfeng/wsdm/ckpt/q4/qwen14b_dare_dslerp_gptq_q4")

In [3]:
from fractions import Fraction

Fraction(3,4).numerator

3